# Structural Processing with Micapipe begins!

## This section explains how to run the proc_structural workflow, how the SLURM submission works, how to interpret each part of the script, and where to find your outputs.

*Micapipe’s structural pipeline provides a comprehensive and standardized framework for transforming raw T1-weighted MRI data into high-quality cortical surfaces, anatomical segmentations, and morphometric features. This processing stream integrates established neuroimaging tools with optimized workflows to ensure reproducibility and reliability across participants and studies.*

### Now lets break down example command for running proc_structural on a SLURM: all the ingredients that we need are

In [ ]:
#!/bin/bash

#SBATCH -c 10
#SBATCH --mem 15G
#SBATCH --time 5:00:00
#SBATCH --job-name struct
#SBATCH --partition=standard


   1. The first line requests 10 CPU cores for your job (-c10). It tells the cluster that your program should run using 10 parallel threads.
   2. Next is the Memory (-mem 15G). This requests 15 GB of RAM. SLURM will only place your job on a compute node that has at least this much free memory. If your job uses more memory than you requested, it may be killed, so this value is important to set correctly.
   3. Runtime Limit (-time 5:00:00). This sets the maximum allowed runtime for the job to 5 hours. If the job exceeds this limit, the scheduler automatically cancels it.Useful for preventing jobs from running indefinitely.
   4. Job Name (-job-name struct). This assigns a custom name to your job—for example, “struct”. It helps you easily identify your job when checking the queue or your job history.
   5. Partition (-partition=standard) This selects the cluster partition (queue) where the job should run. Clusters usually offer partitions like “standard”, “high-memory”, “GPU”, etc. The partition controls priority, hardware type, and resource limits. The higher priority, the faster cluster starts the submitted job.


In [ ]:
# cores
export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK

This export command ensures that your software uses the same number of threads as the CPU cores you requested.

In [ ]:
# define tmp
job_tmp_dir=$TMPDIR/$SLURM_JOB_ID
mkdir $job_tmp_dir

The Temporary Working Directory and The SLURM. *(if you are not familiar with the SLURM you can find information in the helpbook document mention in the README.md file of this repo)* The tmp line defines the temporary working directory on the compute node. This is required on SLURM because each compute node has its own local space. Using local scratch improves speed and prevents network storage overload

In [ ]:
# params
sub=$1
ses=$2

Parameters define the **subject** and **session** labels in your BIDS dataset.

In [ ]:
micapipe_img=/data/p_SoftwareServiceLinux_micapipe/0.2.3/singularity/
bids=/data/your/BIDS
out=/data/your/BIDS/derivatives
fs_lic=/data/your/resources/license.txt

micapipe_img = This is the Micapipe Singularity/Apptainer image, e.g.: micapipe_v0.2.3.sif

*Why you need it*: Contains all required software (FreeSurfer, FSL, ANTs, MRtrix3) Guarantees reproducibility. Ensures the correct Micapipe version. How to get it: Download from the official release page. Or build it locally using the Micapipe recipe. Potential error: If the image is missing or not correctly bound, the job will fail before structural processing begins.

In [ ]:
# run
singularity run --cleanenv --writable-tmpfs --containall \
    -B ${bids}:/bids \
	-B ${out}:/out \
	-B ${job_tmp_dir}:/tmp \
	-B ${fs_lic}:/opt/licence.txt \
	${micapipe_img} \
	-bids /bids -out /out -fs_licence /opt/licence.txt -threads ${OMP_NUM_THREADS} -sub ${sub} -ses ${ses} \
    -proc_structural

# Clean up the temporary data at the end of the job.
rm -fr $job_tmp_dir

check_ComputeClusterSlurm_memory-usage

### Notice the last line -proc_structural

It is crucial to have this line of command in order to run the structural processing with Micapipe

Now we have saved the command line above in the file named: *proc_struct.sh*; When constructing the wrapper for this script notice where you need to adpt to your path


# This is a wrapper that will call the command line above:

In [ ]:
#!/bin/bash

# create directory to save log files
logs_dir="/data/your/path/to/logs"

#run job
sub="0001"
ses="01"

sbatch --error "${logs_dir}/${sub}_${ses}".err \
    --output "${logs_dir}/${sub}_${ses}".out \
    /data/your/scripts/proc_struct.sh  "${sub}" "${ses}"

Adapt the last line of code to match the path to your script and its name

## In order to run the script we have constructed above (you can see the script used for the Spacetop Dataset preprocessing in the scripts folder of the repo) What we do from now is that we send our wrapper script to a submission node.

Please move on to the next Step 2 tutorial SLURM